# Regresión Logística

In [5]:
import os
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

## sklearn - metrics
* precision_score : Calculo la precisión del modelo
* accuracy_score : Calculo la exactitud del modelo
* recall_score : Calculo la sensibilidad del modelo
* f1_score : Calculo el Puntaje F1 del modelo
* roc_auc_score : Calculo la curva ROC - AUC del modelo